# **Logbook es. 10**

## **Introduzione**

Con la seguente esperienza si volgiono esplorare capacità e funzionamento del sensore inerziale fornitoci per la pauda invernale.

### **Task 1**

Obiettivo: Scegliere due canali digitali a piacere fra i 16 disponibili con nomi che vanno da DIO0 (digital input/output numero 0) a DIO15 (digital input/output numero 15), in seguito i canali scelti verranno indicati con A e B. Dopodichè eseguire le sequenti azioni:

* Selezionare la modalità Patterns (generatore di segnali digitali) e aggiungere un singolo segnale su A, selezionare output DD, tipo Random, una frequenza di 500Hz e avviare la generazione con il tasto RUN;

* Cortocircuitare A con uno degli ingressi analogici di Analog Discovery 2, per esempio Ch1 (ricordare la terra!), e verificare la corretta generazione di un segnale di tipo digitale;

* Selezionare la modalità Logic (oscilloscopio digitale) e definire un bus a cui aggiungere il secondo canale digitale scelto B: scollegare A dall’ingresso analogico e collegarlo a B per verificare la corretta misura
ed identificazione dei livelli digitali.